# Munder Difflin Multi-Agent System: Simple Project Report

## 1. How the System Works

We built this system like a small office with a manager and two specialized workers. This "Manager-Worker" setup stops any one part of the code from getting overwhelmed.

### The Team:

* **The Manager (Orchestrator)**: This is the "brain." It reads what the customer wants and decides which worker should handle the job.
* **The Inventory Worker**: This worker checks the warehouse. They use a tool to see if paper is in stock or if they need to order more from a supplier.
* **The Sales Worker**: This worker handles the money. They check past prices to keep things fair and record every sale in the database.

---

## 2. Testing Results

Tested the system with 20 different customer requests.

### What Went Well

* **Smart Planning**: The agents were smart enough to realize that if a customer wanted paper in 2025, but the supplier couldn't deliver until 2026, they should warn the customer.

### What Needs Fixing

* **The date Error**: When recording sales, the system used the actual current date (2026) instead of the "simulation" date (2025). This made the financial reports look like nothing was happening in 2025.

## 3. How to Make it Better

To get this ready for real-world use, I recommend these changes:

1. **Fix the Dates**: Change the tools so the agents can log a sale on a specific past date instead of just using "right now".
